# Coulomb gauge transformation

### Simulation parameters

In [2]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 256   
tau_sim = 0.2     
DTS = 8     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

nevents = 1

### Environment variables

In [4]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="3"

os.environ["MY_NUMBA_TARGET"] = "cuda"
# os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
import curraun.lattice as latt
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
# Number of colors
Nc = su.NC
# Dimension of algebra 
Dg = su.GROUP_ELEMENTS

### Extract gauge field from gauge link

In [5]:
def ln_ah(ui):
    buf0 = su.dagger(ui)
    buf1 = su.mul_s(buf0, -1)
    buf2 = su.add(ui, buf1)
    # buf3 = su.mul_s(buf2, -0.5j)
    buf3 = su.mul_s(buf2, 0.5)
    ai = np.array(buf3)
    return ai

### Simulate glasma

In [8]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(): 

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Transverse gauge links
    ax = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))
    ax_taylor = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Copy gauge links and gauge fields from the memory of the GPU
            if t%DTS == 0:
                ui = s.u1.copy()
                ax[t//DTS] = np.apply_along_axis(ln_ah, 1, ui[:, 0, :])
                        
            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    return ax

### Run simulations

In [9]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')